### Stock Price Prediction with LSTM
Using LSTM is one of the best machine learning approaches for time series forecasting. LSTMs are recurrent neural networks designed to remember data for a longer period. So, whenever you are working on a problem where your neural network fails to memorize data, you can use LSTM neural network. You can read more about LSTMs here.

Now in this section, I will take you through the task of Stock price prediction with LSTM using the Python programming language. I will start this task by importing all the necessary Python libraries and collecting the latest stock price data of Apple:

In [ ]:
# !pip install yfinance

In [8]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()


d1 = today.strftime("%Y-%m-%d")
# The strftime() method returns a string representing date and time using date, time or datetime object.
end_date = d1
d2 =date.today() -timedelta(days=5000)
d2 = d2.strftime("%Y-%m-%d")

start_date =d2


data =yf.download('AAPL', start=start_date,
                 end=end_date,
                 progress=False)
data["Date"] = data.index
data = data[["Date","Open","High","Low","Close","Adj Close", "Volume"]]
data.reset_index(drop=True, inplace= True)
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
3439,2023-02-27,147.710007,149.169998,147.449997,147.919998,147.919998,44998500
3440,2023-02-28,147.050003,149.080002,146.830002,147.410004,147.410004,50547000
3441,2023-03-01,146.830002,147.229996,145.009995,145.309998,145.309998,55479000
3442,2023-03-02,144.380005,146.710007,143.899994,145.910004,145.910004,52238100
3443,2023-03-03,148.039993,151.110001,147.330002,151.029999,151.029999,70668500


#### A candlestick chart gives a clear picture of the increase and decrease in stock prices, so let’s visualize a candlestick chart of the data before moving further:

In [11]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"], 
                                        high=data["High"],
                                        low=data["Low"], 
                                        close=data["Close"])])
figure.update_layout(title = "Apple Stock Price Analysis", 
                     xaxis_rangeslider_visible=False)
figure.show()

In [ ]:
# Now let’s have a look at the correlation of all the columns with the Close column as it is the target column:

In [12]:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
Adj Close    0.999930
Low          0.999874
High         0.999871
Open         0.999722
Volume      -0.518019
Name: Close, dtype: float64


# Training LSTM for Stock Price Prediction


Now I will start with training an LSTM model for predicting stock prices. I will first split the data into training and test sets:



In [17]:
x =data[["Open","High","Low","Volume"]]
print(x)
y = data["Close"]
x = x.to_numpy()
print(x)
y = y.to_numpy()
print(y)

y = y.reshape(-1,1)

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.2,random_state=42)

            Open        High         Low     Volume
0       5.123571    5.141071    5.055000  567616000
1       5.092143    5.135714    5.064286  434224000
2       5.125000    5.166429    5.090000  414178800
3       5.044643    5.101071    4.992500  370479200
4       4.953571    4.963929    4.866071  498688400
...          ...         ...         ...        ...
3439  147.710007  149.169998  147.449997   44998500
3440  147.050003  149.080002  146.830002   50547000
3441  146.830002  147.229996  145.009995   55479000
3442  144.380005  146.710007  143.899994   52238100
3443  148.039993  151.110001  147.330002   70668500

[3444 rows x 4 columns]
[[5.12357092e+00 5.14107084e+00 5.05499983e+00 5.67616000e+08]
 [5.09214306e+00 5.13571405e+00 5.06428623e+00 4.34224000e+08]
 [5.12500000e+00 5.16642904e+00 5.09000015e+00 4.14178800e+08]
 ...
 [1.46830002e+02 1.47229996e+02 1.45009995e+02 5.54790000e+07]
 [1.44380005e+02 1.46710007e+02 1.43899994e+02 5.22381000e+07]
 [1.48039993e+02 1.51110001e+02

# Now I will prepare a neural network architecture for LSTM:

In [18]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True , input_shape = (xtrain.shape[1],1)))
model.add(LSTM(64,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

2023-03-06 14:33:26.653723: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-06 14:33:26.653753: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-06 14:33:26.653771: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nsl47): /proc/driver/nvidia/version does not exist
2023-03-06 14:33:26.655365: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 128)            66560     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117,619
Trainable params: 117,619
Non-trainable params: 0
_________________________________________________________________


Now here is how we can train our neural network model for stock price prediction:



In [19]:
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(xtrain,ytrain, batch_size=2, epochs =30)

Epoch 1/30
2755/2755 [==============================] - 7s 2ms/step - loss: 274.6550
Epoch 2/30
2755/2755 [==============================] - 5s 2ms/step - loss: 9.8663
Epoch 3/30
2755/2755 [==============================] - 5s 2ms/step - loss: 15.6051
Epoch 4/30
2755/2755 [==============================] - 5s 2ms/step - loss: 12.7422
Epoch 5/30
2755/2755 [==============================] - 5s 2ms/step - loss: 7.6536
Epoch 6/30
2755/2755 [==============================] - 5s 2ms/step - loss: 8.4641
Epoch 7/30
2755/2755 [==============================] - 6s 2ms/step - loss: 7.1794
Epoch 8/30
2755/2755 [==============================] - 6s 2ms/step - loss: 8.0259
Epoch 9/30
2755/2755 [==============================] - 6s 2ms/step - loss: 5.8318
Epoch 10/30
2755/2755 [==============================] - 6s 2ms/step - loss: 7.5959
Epoch 11/30
2755/2755 [==============================] - 6s 2ms/step - loss: 5.9523
Epoch 12/30
2755/2755 [==============================] - 6s 2ms/step - loss: 6.29

Let's test this model by giving input values according to the features that we have used to train this model and predicting this final result :


In [22]:
import numpy as np
#feature = [Open , High, low, Adj Close, Volume]
features = np.array([[177.089996, 180.419998, 177.070007, 74919600]])
model.predict(features)

1/1 [==============================] - 0s 439ms/step


array([[167.34036]], dtype=float32)

So this is how we can use LSTM neural network architecture for the task of stock price prediction.

# Summary
LSTM stands for Long Short Term Memory Networks. It is a recurrent neural network designed to remember data for longer. Using LSTM is one of the best machine learning approaches for time series forecasting. I hope you liked this article on predicting stock prices with LSTM using Python. Feel free to ask valuable questions in the comments section below.